## Prueba de modelos ##

### Version 4: Modelo con 2 DER, planificación anual, granularidad horaria ###

Si el modelo tiene que optimizar el balance por horas, tiene 8640 VD, pero sigue teniendo 2 VD binarias

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import pandapower as pp

In [ ]:
import os

In [ ]:
import pandas as pd

In [ ]:
import pyomo.environ as pe

In [ ]:
import math

In [ ]:
import random

In [ ]:
import itertools

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
import ModelWriters.SimpleBusbar

Variables que define el modelo:

y para el año

d para el día del año

h para la hora del día

dt la granularidad del modelo, en horas


temp la temperatura

wv la velocidad del viento

I la irradiación solar


eg el crecimiento económico en pu respecto al año 0


Estas variables llegan como parámetros en forma de diccionario en el argumento 'model_status' (ver este nombre)

Todas las funciones reciben este argumento, la lógica de la función indica que valor retorna. Pr ejemplo, si la granularidad es 24 h, debe retornar el valor medio del parámetro producido.

## Construcción de Escenarios

In [45]:
import ModelWriters.Scenes.SceneBuilder as SB

In [46]:
s = SB.SceneBuilder(years = 5, growth_rate = 0.025, discount_rate = 0.05)
scenes = s.build_scenes()
scenes

,year,day,dd,hour,dt,growth,discount,solar_irradiance,wind_speed
0,0.0,0.0,1.0,0.0,1.0,1.000000,1.000000,0.0,1.239677
1,0.0,0.0,1.0,1.0,1.0,1.000000,1.000000,0.0,3.479501
2,0.0,0.0,1.0,2.0,1.0,1.000000,1.000000,0.0,4.718673
3,0.0,0.0,1.0,3.0,1.0,1.000000,1.000000,0.0,4.728113
4,0.0,0.0,1.0,4.0,1.0,1.000000,1.000000,0.0,4.939853
...,...,...,...,...,...,...,...,...,...
43795,4.0,364.0,1.0,19.0,1.0,1.103813,0.822702,0.0,6.856482
43796,4.0,364.0,1.0,20.0,1.0,1.103813,0.822702,0.0,7.283973
43797,4.0,364.0,1.0,21.0,1.0,1.103813,0.822702,0.0,6.286363
43798,4.0,364.0,1.0,22.0,1.0,1.103813,0.822702,0.0,5.143877


## Model Tests

In [ ]:
from ModelWriters.Resources.ExtGrid import ExtGrid

In [ ]:
from ModelWriters.Resources.Load import Load

In [ ]:
from ModelWriters.Resources.BaseGen import Generator, DiscreteGenerator

In [ ]:
from ModelWriters.Resources.Storage import Storage

In [ ]:
model = pe.ConcreteModel()

In [ ]:
exg = ExtGrid('EXT')

In [ ]:
bg = Generator('GEN')

In [ ]:
dg = DiscreteGenerator('DGEN')

In [ ]:
load = Load('LOAD')

In [ ]:
storage = Storage('STORAGE')

In [ ]:
exg['pr_mw'] = 5.0
exg['ic_0_mu'] = 1.2
exg['ic_1_mu'] = 0.2
exg['oc_0_mu'] = 0.001
exg['oc_1_mu'] = 0.005

#exg.pa_pu = wind_output

In [ ]:
load['pr_mw'] = .5
load['ic_0_mu'] = 1.2
load['ic_1_mu'] = 0.2
load['oc_0_mu'] = 0.001
load['oc_1_mu'] = 0.005
load['pa_pu'] = Demand.scalonated_seasoned_randomized_demand

#exg.pa_pu = wind_output

In [ ]:
dg['ic_0_mu'] = 0.44
dg['ic_1_mu'] = 0.2
dg['unit_cost_mu'] = 0.3
dg['oc_0_mu'] = 0.001
dg['oc_1_mu'] = 0.005

In [ ]:
exg.initialize_model(model, Escenarios)

In [ ]:
bg['pr_mw'] = 5.0
bg['ic_0_mu'] = 1.2
bg['ic_1_mu'] = 0.2
bg['oc_0_mu'] = 0.001
bg['oc_1_mu'] = 0.005


In [ ]:
bg.initialize_model(model, Escenarios)

In [ ]:
load.initialize_model(model, Escenarios)

In [ ]:
dg.initialize_model(model, Escenarios)

In [ ]:
storage.initialize_model(model, Escenarios)

In [ ]:
model.pprint()

## Line Model

In [ ]:
from ModelWriters.Resources.SimpleLine import SimpleLine

In [ ]:
model = pe.ConcreteModel()

In [ ]:
sl = SimpleLine('Aquí', 'Allá')

In [ ]:
print(sl)

In [ ]:
sl.initialize_model(model, Escenarios)

In [ ]:
model.pprint()

## Line Cap Model

In [32]:
import ModelWriters as mw

In [ ]:
from ModelWriters.Resources.ExtGrid import ExtGrid
from ModelWriters.Resources.Load import Load
from ModelWriters.Resources.PV import PVGenerator
from ModelWriters.Resources.WT import WTGenerator
from ModelWriters.Simulation import ElectricityCost

In [70]:
net = pp.create_empty_network() 
b0 = pp.create_bus(net, vn_kv=13.2)

pp.create_ext_grid(net, name = 'SET', bus=b0)

#ramal 1
b11 = pp.create_bus(net, vn_kv=13.2)
b12 = pp.create_bus(net, vn_kv=13.2)

pp.create_line(net, from_bus=b0, to_bus=b11, length_km=0.8, std_type="NAYY 4x50 SE")
pp.create_line(net, from_bus=b11, to_bus=b12, length_km=2.1, std_type="NAYY 4x50 SE")

pp.create_sgen(net, b11, p_mw=0.075, q_mvar=0.025, max_p_mw=0.15, max_q_mvar=0.15)
pp.create_load(net, bus=b12, p_mw=0.350)


#ramal 2
b21 = pp.create_bus(net, vn_kv=13.2)
pp.create_line(net, from_bus=b0, to_bus=b21, length_km=5.0, std_type="NAYY 4x50 SE")
pp.create_sgen(net, b21, p_mw=0.15, q_mvar=0.00, max_p_mw=0.40, max_q_mvar=0.10)
pp.create_storage(net, bus=b21, p_mw=0.25, q_mvar=0.1, max_e_mwh = 3.0)

#ramal 3
b31 = pp.create_bus(net, vn_kv=13.2)
b32 = pp.create_bus(net, vn_kv=13.2)
pp.create_line(net, from_bus=b0, to_bus=b31, length_km=0.5, std_type="NAYY 4x50 SE")
pp.create_line(net, from_bus=b31, to_bus=b32, length_km=1.5, std_type="NAYY 4x50 SE")
pp.create_sgen(net, b31, p_mw=0.08, q_mvar=0.00, max_p_mw=0.30, max_q_mvar=0.10)
pp.create_load(net, bus=b32, p_mw=0.85, q_mvar=0.33)


#ramal 4
b41 = pp.create_bus(net, vn_kv=13.2)
pp.create_line(net, from_bus=b0, to_bus=b41, length_km=0.5, std_type="NAYY 4x50 SE")
pp.create_load(net, bus=b41, p_mw=0.22, q_mvar=0.05)
pp.create_storage(net, bus=b41, p_mw=0.5, q_mvar=0.1, max_e_mwh = 5.0)


1

In [34]:
def net_add_optfw(net):
    #agregamos parámetros para el modelo de optimización
    tables = [net.ext_grid, net.load, net.sgen, net.storage, net.line]

    for t in tables: 
        #initial cost
        t['model'] = None

net_add_optfw(net)

In [35]:
net.ext_grid

,name,bus,vm_pu,va_degree,in_service,model
0,SET,0,1.0,0.0,True,None


In [36]:
import ModelWriters.Resources as Resources

In [42]:
net.ext_grid['model'][0] = Resources.ExtGrid('SET', pr_mw = 5.0)

net.load['model'][0] = Resources.Load('Tornquinst', pr_mw = net.load['p_mw'][0])
net.load['model'][1] = Resources.Load('Bell Ville', pr_mw = net.load['p_mw'][1])
net.load['model'][2] = Resources.Load('City Bell' , pr_mw = net.load['p_mw'][2])


"""
m = PVGenerator('PV1')
m.ic_0_mu = 0.5
m.ic_1_mu = 0.05
m.oc_0_mu = 0.0
m.oc_1_mu = 400e-6

net.sgen['name'][0] = m.name
net.sgen['model'][0] = m


m = PVGenerator('PV2')
m.ic_0_mu = 3.0
m.ic_1_mu = 0.75
m.oc_0_mu = 0.0
m.oc_1_mu = 500e-6

net.sgen['name'][2] = m.name
net.sgen['model'][2] = m

m = WTGenerator('WT1')
m.ic_0_mu = 5.0
m.ic_1_mu = 3.0
m.oc_0_mu = 0.0
m.oc_1_mu = 700e-6

net.sgen['name'][1] = m.name
net.sgen['model'][1] = m

"""

"""
m = Storage.Storage('ST1')

m.pa_pu = 1.0
m.ic_0_mu = 0.5
m.ic_1_mu = 1.0
m.ic_1_mu_cap = 0.5
m.oc_0_mu = 0.0

net.storage['name'][0] = 'ST1'
net.storage['model'][0] = m


m = Storage.Storage('ST2')
net.storage['name'][1] = 'ST2'
net.storage['model'][1] = m

m.pa_pu = 1.0
m.ic_0_mu = 0.5
m.ic_1_mu = 2.0
m.ic_1_mu_cap = 7.0
m.oc_0_mu = 0.0
m.oc_1_mu = 1400e-6
"""
None


In [69]:
net.ext_grid['model'][0].oc_1_mw

In [67]:
c = 0.0
if not c:
    print("True")
else:
    print("False")

True


In [44]:
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type,model
0,None,2,0.35,0.00,0.0,0.0,NaN,1.0,True,wye,Resource: Tornquinst
1,None,5,0.85,0.33,0.0,0.0,NaN,1.0,True,wye,None
2,None,6,0.22,0.05,0.0,0.0,NaN,1.0,True,wye,None


In [ ]:
net.line

In [ ]:
net.bus

In [ ]:
net.sgen

In [ ]:
from ModelWriters.MultiBusbar import MultiBusbarModelWriter

In [ ]:
mw = MultiBusbarModelWriter(net, Escenarios)

In [ ]:
mw.create_model()

In [ ]:
#mw.model.pprint()

In [ ]:
#solver_manager = pe.SolverManagerFactory('neos')
#result_obj = solver_manager.solve(model.model, opt='cplex')

opt = pe.SolverFactory('cplex')
#opt = pe.SolverFactory('glpk')
result_obj= opt.solve(mw.model, tee=True)

In [ ]:
mw.backconfigure_network()

In [ ]:
net.sgen

In [ ]:
Resultados = mw.get_scenes_results()

In [ ]:
# Set the plot curve with markers and a title
fig, ax = plt.subplots(1, 1)

fig.set_size_inches(10,6)
##a=180*24
#b=360
##b=180*24+5*24
a=0
b=24
for col in Resultados:
    ax.plot(Resultados[col][a:b], label = col)

#ax.plot(Resultados['Tornquinst'][a:b]+Resultados['Bell Ville'][a:b]+Resultados['City Bell'][a:b], label = 'Load')

#ax.plot(Resultados['time'][240:360], Resultados['p_mw_PV'][240:360], label = 'PV')
#ax.plot(Resultados['time'][240:360], Resultados['p_mw_Load'][240:360], label = 'Load')

ax.set_xlabel('time')
ax.set_ylabel('P[mw]')

ax.legend()

ax.grid(True)


plt.show()

In [ ]:

Resultados